In [ ]:
from datasets import load_dataset

dataset = load_dataset("nyu-mll/glue", "qnli")

'''
# you can use any of the following config names as a second argument:
"ax", "cola", "mnli", "mnli_matched", 
"mnli_mismatched", "mrpc", "qnli", "qqp", 
"rte", "sst2", "stsb", "wnli"
'''

In [ ]:
from transformers import BertTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
dataset['train'].shape 

(104743, 4)

In [6]:
# dataset.set_format(type='torch', columns=['sentence', 'label'])

In [4]:
dataset['train'][0:2]

{'question': ['When did the third Digimon series begin?',
  'Which missile batteries often have individual launchers several kilometres from one another?'],
 'sentence': ['Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.',
  'When MANPADS is operated by specialists, batteries may have several dozen teams deploying separately in small sections; self-propelled air defence guns may deploy in pairs.'],
 'label': [1, 1],
 'idx': [0, 1]}

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

In [28]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
})

In [9]:
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

In [25]:
dataset['train'][0:2]['sentence']

['Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.',
 'When MANPADS is operated by specialists, batteries may have several dozen teams deploying separately in small sections; self-propelled air defence guns may deploy in pairs.']

In [27]:
tokenizer.batch_encode_plus(dataset['train'][0:2]['sentence'])

{'input_ids': [[101, 45227, 10105, 10551, 21396, 11360, 10271, 10111, 10992, 10108, 10105, 21396, 10189, 15689, 117, 83306, 27324, 10901, 19135, 169, 25100, 10165, 10111, 10798, 13486, 29025, 23068, 10114, 10474, 13617, 17846, 83306, 10479, 10149, 10472, 74720, 15983, 36884, 10662, 10455, 50438, 10111, 10798, 17217, 15092, 13405, 10106, 10105, 11364, 13847, 119, 102], [101, 12242, 73020, 52099, 33953, 10124, 19301, 10155, 64956, 10107, 117, 72781, 11387, 10529, 11736, 84956, 14404, 10104, 91823, 10230, 91614, 10106, 12474, 30293, 132, 16567, 118, 30045, 14000, 10162, 12566, 47245, 31199, 11387, 10104, 91823, 10106, 55736, 119, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
# thu 2 cach implement khac nhau
# Case 1: using output hidden state cua question encoder is input of context encoder, then use the output of context encoder to predict the label
# Case 2: using output hidden state of context encoder is the input to question encoder, then use the output of question encoder to predict the label

In [7]:
len(dataset['train'])

104743

In [30]:
dataset['train'][0]

{'question': 'When did the third Digimon series begin?',
 'sentence': 'Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese.',
 'label': 1,
 'idx': 0}

In [23]:
tokenizer(dataset['train'][0:2]['question'], padding=True, truncation=True, return_tensors='pt')

{'input_ids': tensor([[  101, 12242, 12172, 10105, 12628, 83306, 11366, 16135,   136,   102,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  101,   160, 39187, 10237, 45174, 72781, 12899, 10529, 16080, 29227,
         10901, 11736, 20127, 10188, 10464, 12864,   136,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [48]:
class SquadDataset(torch.utils.data.Dataset):
	'''
	- Creates batches dynamically by padding to the length of largest example
	  in a given batch.
	- Calulates character vectors for contexts and question.
	- Returns tensors for training.
	'''
	
	def __init__(self, data, batch_size, tokenizer):
		
		self.batch_size = batch_size
		data = [data[i:i+self.batch_size] for i in range(0, len(data), self.batch_size)]
		self.data = data
		self.tokenizer = tokenizer
		
		
	def __len__(self):
		return len(self.data)
	
	def __iter__(self):
		'''
		Creates batches of data and yields them.
		
		Each yield comprises of:
		:padded_context: padded tensor of contexts for each batch 
		:padded_question: padded tensor of questions for each batch 
		:label: 
		
		'''
		
		for batch in self.data:
			question = self.tokenizer(batch['question'], padding=True, truncation=True, return_tensors='pt', max_length=96)['input_ids']
			context = self.tokenizer(batch['sentence'], padding=True, truncation=True, return_tensors='pt', max_length=256)['input_ids']
			label = torch.BoolTensor(batch['label'])
			
			yield question, context, label
			
			

In [49]:
len(dataset['train'])

104743

In [50]:
squad = SquadDataset(dataset['train'], 8, tokenizer)

In [ ]:
for i in squad:
    print(i)
    break

In [52]:
train_data = torch.utils.data.DataLoader(squad, batch_size=8, shuffle=True)

In [ ]:
class RNNqnli(nn.Module):
	def __init__(self, question_size, context_size, hidden_size, num_layers, num_classes):
		super(RNNqnli, self).__init__()
		self.hidden_size = hidden_size # hidden size of RNN
		self.num_layers = num_layers	# number of layers of RNN
		self.rnn_question = nn.RNN(question_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
		# output of question encoder [batch_size, seq_len, hidden_size * 2]

		self.rnn_context = nn.RNN(context_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
		# output of context encoder [batch_size, seq_len, hidden_size * 2]
		
		self.fc = nn.Linear(hidden_size, num_classes)

	def forward(self, x):
		
		


In [1]:
torch.max(torch.Tensor([[1,2,3], [4,5,6]]), dim=-1)

NameError: name 'torch' is not defined

In [27]:
s = torch.randn(2,3,4)

In [28]:
import torch.nn.functional as F

In [33]:
torch.max(s, dim=-1)[0].shape

torch.Size([2, 3])

In [31]:
b = F.softmax(torch.max(s, dim=2)[0], dim=1)

In [32]:
b.shape

torch.Size([2, 3])

In [6]:
import torch

In [8]:
torch.randn(1,2,3).unsqueeze(2).shape

torch.Size([1, 2, 1, 3])

In [1]:
list_string = ['this', 'is', 'a', 'text']

In [2]:
[list(a) for a in list_string ]

[['t', 'h', 'i', 's'], ['i', 's'], ['a'], ['t', 'e', 'x', 't']]

In [7]:
import torch.nn as nn
import torch

In [8]:
encoder_layer = nn.TransformerEncoderLayer(d_model=48, nhead=4, batch_first=True)
src = torch.rand(5, 3, 48)
out = encoder_layer(src)

In [9]:
out.shape

torch.Size([5, 3, 48])

In [15]:
a = torch.Tensor([[1,2,3], [4,5,6], [7,8,9]])
b = torch.Tensor([[1,1,1], [1,1,1], [1,1,1]])
c = torch.mul(a,b)
print(a)
print(c)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [3]:
x = lambda:[0,0,0,0]
y = x()

In [4]:
y

[0, 0, 0, 0]

In [1]:
from collections import deque

In [3]:
queue = deque([(0, -1)])

In [4]:
queue

deque([(0, -1)])

In [6]:
def min_cost_to_paint_tree(N, edges):
    from collections import defaultdict, deque
    
    # Create adjacency list for the tree
    tree = defaultdict(list)
    for u, v in edges:
        tree[u].append(v)
        tree[v].append(u)
    
    # BFS to calculate the minimum cost to paint all vertices
    def bfs(start):
        visited = set()
        queue = deque([(start, -1)])  # (current_node, parent_node)
        moves = 0
        
        while queue:
            node, parent = queue.popleft()
            visited.add(node)
            parent = node
            for neighbor in tree[node]:
                if neighbor not in visited:
                    queue.append((neighbor, node))
                    if parent != -1:
                        moves += 1  # Action 2 (moving piece) costs 1
        
        return moves
    
    # We can start BFS from any node, let's choose node 1
    return bfs(1)

# Example 1
N = 4
edges = [(1, 2), (1, 3), (1, 4)]
print(min_cost_to_paint_tree(N, edges))  # Output: 3


3


In [2]:
import torch

In [5]:
a = torch.Tensor([[[1,2,3], [4,5,6], [7,8,9]], [[1,2,3], [4,5,6], [7,8,9]]])
a.mean(dim=1)

tensor([[4., 5., 6.],
        [4., 5., 6.]])

In [7]:
a.shape

torch.Size([2, 3, 3])